In [1]:
import numpy as np
import sympy as sp
sp.init_printing() 

Задаем значения из своего варианта

In [2]:
A = sp.Matrix(np.array([
    [ 1,  5,  7],
    [ 0,  3,  5],
    [ 0,  2,  1],
]))
B = sp.Matrix(np.array([
    [ 2, -3],
    [ 1,  2],
    [ 1,  1]
]))
lambd = sp.Matrix(np.array([
    [-3],
    [-1],
    [-3],
]))
r = sp.Matrix([
    [ 2],
    [ 3],
])

In [3]:
S = B.row_join(A @ B).row_join(A @ A @ B)
print("Матрица управляемости:")
S

Матрица управляемости:


⎡2  -3  14  14  75  104⎤
⎢                      ⎥
⎢1  2   8   11  39  58 ⎥
⎢                      ⎥
⎣1  1   3   5   19  27 ⎦

Тут, вообще говоря, не совсем понятно, что имеется ввиду под определителем неквадратной матрицы... Судя по методичке и автотестам, определитель на первых трех столбцах...

In [4]:
detS = S[:, :3].det()
print("Определитель матрицы управляемости det(S) = ", detS)
print("Система управляема") if detS != 0 else print("Система не управляема")

Определитель матрицы управляемости det(S) =  -33
Система управляема


In [5]:
lam = sp.symbols('lambda')

F = sp.Matrix([
    [lam - 2, -1, -4],
    [0, lam - 2, 3],
    [0, 0, lam - 5]
])

def compute_D2(matrix):
    minors_2x2 = []
    n = matrix.shape[0]
    
    # Проходим по всем возможным подматрицам 2x2
    for i in range(n):
        for j in range(n):
            submatrix = matrix.minor_submatrix(i, j)  # Удаляем i-ю строку и j-й столбец
            if submatrix.shape == (2, 2):
                minors_2x2.append(submatrix.det())  # Считаем определитель миноров 2-го порядка

    # Находим НОД всех найденных определителей
    D_2 = sp.gcd(minors_2x2)
    return D_2

D_3 = F.det()
D_2 = compute_D2(F)
D_1 = sp.gcd(list(F))
D_0 = 1

E_3 = D_3 / D_2
E_2 = D_2 / D_1
E_1 = D_1 / D_0

def is_non_trivial(polynomial):
    return polynomial != 1

def minimal_control_dimension(E_3, E_2, E_1):
    non_trivial_polynomials = [E_3, E_2, E_1]
    s = sum(1 for poly in non_trivial_polynomials if is_non_trivial(poly))
    return s

s = minimal_control_dimension(E_3, E_2, E_1)

print(f"s = {s}")
if s == 1:
    print("Допускает скалярное управление")
elif s == 0:
    print("Система не управляема")
else:
    print("Не допускает скалярное управление")

s = 1
Допускает скалярное управление


In [6]:
c = A.charpoly().all_coeffs()
c = sp.Matrix(np.flip(c[1:]).reshape((-1, 1)))
print("Коэффициенты характеристического полинома разомкнутой системы:")
c

Коэффициенты характеристического полинома разомкнутой системы:


⎡7 ⎤
⎢  ⎥
⎢-3⎥
⎢  ⎥
⎣-5⎦

In [7]:
sigma_1 = np.sum(lambd)
sigma_2 = lambd[0] * lambd[1] + lambd[0] * lambd[2] + lambd[1] * lambd[2]
sigma_3 = np.prod(lambd)

cx = np.zeros(3, dtype=int)
cx[0] = sigma_3 * (-1) ** (3 - 0)
cx[1] = sigma_2 * (-1) ** (3 - 1)
cx[2] = sigma_1 * (-1) ** (3 - 2)

cx = sp.Matrix(cx.reshape((-1, 1)))

print("Коэффициенты характеристического полинома замкнутой системы:")
cx

Коэффициенты характеристического полинома замкнутой системы:


⎡9 ⎤
⎢  ⎥
⎢15⎥
⎢  ⎥
⎣7 ⎦

In [8]:
b = B @ r

d3 = b
d2 = A @ d3 + c[2] * b
d1 = A @ d2 + c[1] * b
D = np.hstack((d1, d2, d3))
D = sp.Matrix(D)
D_1 = D.inv()
print("Матрица D^-1:")
D_1

Матрица D^-1:


⎡       -65   115 ⎤
⎢1/217  ────  ────⎥
⎢       2387  2387⎥
⎢                 ⎥
⎢        90   -19 ⎥
⎢1/217  ────  ────⎥
⎢       2387  341 ⎥
⎢                 ⎥
⎢        59   394 ⎥
⎢1/217  ────  ────⎥
⎣       2387  2387⎦

In [9]:
v = (c - cx).T @ D_1 
print("Скалярное управление:")
v

Скалярное управление:


⎡-32   -314   -2564 ⎤
⎢────  ─────  ──────⎥
⎣217    341    2387 ⎦

In [10]:
u = r @ v
print("Векторное управление:")
u

Векторное управление:


⎡-64   -628   -5128 ⎤
⎢────  ─────  ──────⎥
⎢217    341    2387 ⎥
⎢                   ⎥
⎢-96   -942   -7692 ⎥
⎢────  ─────  ──────⎥
⎣217    341    2387 ⎦